# AMV with HadISST

In [1]:
import numpy as np
from numpy import *
import bisect
import scipy as sp
from scipy.stats import gaussian_kde, ttest_ind
import numpy.matlib 
import math
from scipy import signal
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
import cartopy as cart
import cartopy.crs as ccrs
from datetime import *
import datetime as datet
from dateutil.relativedelta import relativedelta, MO
import calendar
import matplotlib.animation as animation
from matplotlib import rc
from dask.distributed import Client, LocalCluster
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
import os
import warnings

In [16]:
def butter_lowpass_filter(data, cutoff_freq=1/11, sample_rate=1, order=4):
    """
    Apply Butterworth Filter 
    
    data - Input data
    cutoff_freq = 1/11 - 11 year filter - the critical frequency or frequencies
    sample_rate = 1 - assuming data is annual 
    order = 2 - The order of the filter
    
    """
    b, a = signal.butter(order, cutoff_freq, btype='low', analog=False)
    y = signal.filtfilt(b, a, data)
    return y

In [17]:
# Load HadISST data set
HadISST= xr.open_dataset('/work/bm1398/m301111/Observations/HadISST/HadISST_sst_1870-2023_1deg.nc')

## HadISST

In [12]:
# Define cos(lat) weights
weights = np.cos(np.deg2rad(HadISST.latitude)) 
# Data masked
HadISST_masked =HadISST.sst.where(HadISST.sst != -1000.)
# Calc monthly mean anomalies
HadISST_mon_anom = HadISST_masked.groupby('time.month') - HadISST_masked.groupby('time.month').mean('time')
# Calc annual means
HadISST_ann_anom = HadISST_mon_anom.groupby('time.year').mean(dim='time')
# Calc weighted NA area average (0-60°N, 7.5 - 75°W) of both annual means and monthly mean anomalies
HadISST_NAav = HadISST_ann_anom.sel(longitude=slice(-75.5,-7.5), latitude=slice(60.5,0.5)).weighted(weights).mean(('longitude','latitude'), skipna=True).sel(year=slice(1870,2022))
HadISST_mon_NAav = HadISST_mon_anom.sel(longitude=slice(-75.5,-7.5), latitude=slice(60.5,0.5)).weighted(weights).mean(('longitude','latitude'), skipna=True).sel(time=slice('1870-01-01','2022-12-01'))
# Calc Butterworth Filter
HadISST_AMV_Index_lpf = xr.DataArray(data=(butter_lowpass_filter(HadISST_NAav)), dims=["year"], coords=dict(time=HadISST_NAav.year))
HadISST_mon_AMV_Index_lpf = xr.DataArray(data=(butter_lowpass_filter(HadISST_mon_NAav)), dims=["time"], coords=dict(time=HadISST_mon_NAav.time))
# Detrend SST data (also removes long term mean)^
HadISST_NAav_lpf_de = xr.DataArray(data=(signal.detrend(HadISST_AMV_Index_lpf)), dims=["year"], coords=dict(year=HadISST_NAav.year))
HadISST_mon_NAav_lpf_de = xr.DataArray(data=(signal.detrend(HadISST_mon_AMV_Index_lpf)), dims=["time"], coords=dict(year=HadISST_mon_AMV_Index_lpf.time))

### Store HadISST AMV for further analysis performed in HighResMIP_ERA5_AMV.ipynb

In [8]:
%store HadISST_NAav_lpf_de
%store HadISST_mon_NAav_lpf_de

Stored 'HadISST_NAav_lpf_de' (DataArray)
Stored 'HadISST_mon_NAav_lpf_de' (DataArray)
